# Garbage Collection

In [1]:
import torch as th

In [2]:
import syft as sy

In [3]:
hook = sy.TorchHook(th)

In [4]:
bob = sy.VirtualWorker(hook,id="bob")

In [5]:
x = th.tensor([1,2,3,4]).send(bob)

In [6]:
bob._objects

{43847014620: tensor([1, 2, 3, 4])}

In [7]:
del x

In [8]:
bob._objects

{}

In [9]:
for i in range(1000):
    x = th.tensor([1,2,3,4]).send(bob)

In [10]:
bob._objects

{39366076984: tensor([1, 2, 3, 4])}

In [11]:
del x

In [12]:
bob._objects

{}

# Toy Federated Learning

In [13]:
from torch import nn, optim

In [14]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad = True)

In [15]:
model = nn.Linear(2,1)

In [16]:
opt = optim.SGD(params=model.parameters(),lr=0.1)

In [17]:
def training(iterations):
    for iter in range(iterations):
        opt.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        opt.step()
        print(loss.data)
training(20)

tensor(13.6964)
tensor(1.4899)
tensor(0.4395)
tensor(0.2685)
tensor(0.1907)
tensor(0.1387)
tensor(0.1016)
tensor(0.0749)
tensor(0.0555)
tensor(0.0412)
tensor(0.0308)
tensor(0.0231)
tensor(0.0173)
tensor(0.0131)
tensor(0.0099)
tensor(0.0075)
tensor(0.0057)
tensor(0.0043)
tensor(0.0033)
tensor(0.0025)


In [18]:
naufil = sy.VirtualWorker(hook, id="naufil")
faizan = sy.VirtualWorker(hook, id="faizan")

In [19]:
data_naufil = data[0:2].send(naufil)
data_faizan = data[2:4].send(faizan)

In [20]:
target_naufil = target[0:2].send(naufil)
target_faizan = target[2:4].send(faizan)

In [21]:
datasets = [(data_naufil,target_naufil),(data_faizan,target_faizan)]

In [22]:
model = nn.Linear(2,1)

In [23]:
opt = optim.SGD(params = model.parameters(), lr = 0.1)

In [24]:
_data, _target = datasets[0]

In [25]:
_data.location

<VirtualWorker id:naufil #tensors:2>

In [26]:
model = model.send(_data.location)

RuntimeError: set_storage is not allowed on Tensor created from .data or .detach()

# Advanced Remote Execution

In [63]:
naufil.clear_objects
faizan.clear_objects

<bound method BaseWorker.clear_objects of <VirtualWorker id:faizan #tensors:4>>

In [53]:
x = th.tensor([1,2,3,4]).send(naufil).send(faizan)

In [54]:
x

(Wrapper)>[PointerTensor | me:92732758512 -> faizan:75315879446]

In [55]:
faizan._objects

{23613880314: (Wrapper)>[PointerTensor | faizan:23613880314 -> naufil:83904550070],
 47735554887: tensor([[1, 0],
         [0, 0]]),
 76176561640: tensor([[1, 0],
         [0, 0]]),
 75315879446: (Wrapper)>[PointerTensor | faizan:75315879446 -> naufil:67688684810]}

In [56]:
naufil._objects

{83904550070: tensor([1, 2, 3, 4]), 49095176888: tensor([[1, 1],
         [0, 1]]), 36408048950: tensor([[1, 1],
         [0, 1]]), 67688684810: tensor([1, 2, 3, 4])}

In [57]:
del x

In [58]:
naufil._objects

{83904550070: tensor([1, 2, 3, 4]), 49095176888: tensor([[1, 1],
         [0, 1]]), 36408048950: tensor([[1, 1],
         [0, 1]]), 67688684810: tensor([1, 2, 3, 4])}

In [59]:
faizan._objects

{23613880314: (Wrapper)>[PointerTensor | faizan:23613880314 -> naufil:83904550070],
 47735554887: tensor([[1, 0],
         [0, 0]]),
 76176561640: tensor([[1, 0],
         [0, 0]]),
 75315879446: (Wrapper)>[PointerTensor | faizan:75315879446 -> naufil:67688684810]}

In [21]:
x

NameError: name 'x' is not defined